In [1]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql import SparkSession
#create Spark session
appName = "Sentiment Analysis in Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
#read csv file into dataFrame with automatically inferred schema
tweets_csv = spark.read.csv('tweets.csv', inferSchema=True, header=True)
tweets_csv.show(truncate=False, n=3)

+------+---------+---------------+---------------------------------+
|ItemID|Sentiment|SentimentSource|SentimentText                    |
+------+---------+---------------+---------------------------------+
|1038  |1        |Sentiment140   |that film is fantastic #brilliant|
|1804  |1        |Sentiment140   |this music is really bad #myband |
|1693  |0        |Sentiment140   |winter is terrible #thumbs-down  |
+------+---------+---------------+---------------------------------+
only showing top 3 rows



In [3]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("SentimentText", col("Sentiment").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+---------------------------------+-----+
|SentimentText                    |label|
+---------------------------------+-----+
|that film is fantastic #brilliant|1    |
|this music is really bad #myband |1    |
|winter is terrible #thumbs-down  |0    |
|this game is awful #nightmare    |0    |
|I love jam #loveit               |1    |
+---------------------------------+-----+
only showing top 5 rows



In [4]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.7, 0.3]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1348 ; Testing data rows: 584


In [5]:
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False, n=5)

+---------------------------------+-----+---------------------------------------+
|SentimentText                    |label|SentimentWords                         |
+---------------------------------+-----+---------------------------------------+
|I adore cheese #brilliant        |1    |[ı, adore, cheese, #brilliant]         |
|I adore cheese #favorite         |1    |[ı, adore, cheese, #favorite]          |
|I adore cheese #loveit           |1    |[ı, adore, cheese, #loveit]            |
|I adore classical music #bestever|1    |[ı, adore, classical, music, #bestever]|
|I adore classical music #loveit  |1    |[ı, adore, classical, music, #loveit]  |
+---------------------------------+-----+---------------------------------------+
only showing top 5 rows



In [7]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=5)

+---------------------------------+-----+---------------------------------------+---------------------------------------+
|SentimentText                    |label|SentimentWords                         |MeaningfulWords                        |
+---------------------------------+-----+---------------------------------------+---------------------------------------+
|I adore cheese #brilliant        |1    |[ı, adore, cheese, #brilliant]         |[ı, adore, cheese, #brilliant]         |
|I adore cheese #favorite         |1    |[ı, adore, cheese, #favorite]          |[ı, adore, cheese, #favorite]          |
|I adore cheese #loveit           |1    |[ı, adore, cheese, #loveit]            |[ı, adore, cheese, #loveit]            |
|I adore classical music #bestever|1    |[ı, adore, classical, music, #bestever]|[ı, adore, classical, music, #bestever]|
|I adore classical music #loveit  |1    |[ı, adore, classical, music, #loveit]  |[ı, adore, classical, music, #loveit]  |
+-----------------------

In [8]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+------------------------------+------------------------------------------------------+
|label|MeaningfulWords               |features                                              |
+-----+------------------------------+------------------------------------------------------+
|1    |[ı, adore, cheese, #brilliant]|(262144,[1689,45361,100089,231588],[1.0,1.0,1.0,1.0]) |
|1    |[ı, adore, cheese, #favorite] |(262144,[1689,100089,108624,231588],[1.0,1.0,1.0,1.0])|
|1    |[ı, adore, cheese, #loveit]   |(262144,[1689,100089,231588,254974],[1.0,1.0,1.0,1.0])|
+-----+------------------------------+------------------------------------------------------+
only showing top 3 rows



In [10]:
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.01)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


In [11]:
tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+-----+------------------------------+-----------------------------------------------------+
|Label|MeaningfulWords               |features                                             |
+-----+------------------------------+-----------------------------------------------------+
|1    |[ı, adore, cheese, #bestever] |(262144,[1689,91011,100089,231588],[1.0,1.0,1.0,1.0])|
|1    |[ı, adore, cheese, #thumbs-up]|(262144,[1689,88825,100089,231588],[1.0,1.0,1.0,1.0])|
+-----+------------------------------+-----------------------------------------------------+
only showing top 2 rows



In [12]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
predictionFinal.show(n=100, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, 
      ", accuracy:", correctPrediction/totalData)

+--------------------------------------------+----------+-----+
|MeaningfulWords                             |prediction|Label|
+--------------------------------------------+----------+-----+
|[ı, adore, cheese, #bestever]               |1.0       |1    |
|[ı, adore, cheese, #thumbs-up]              |1.0       |1    |
|[ı, adore, cheese, #toptastic]              |1.0       |1    |
|[ı, adore, classical, music, #brilliant]    |1.0       |1    |
|[ı, adore, classical, music, #favorite]     |1.0       |1    |
|[ı, adore, classical, music, #thumbs-up]    |1.0       |1    |
|[ı, adore, classical, music, #toptastic]    |1.0       |1    |
|[ı, adore, coffee, #brilliant]              |1.0       |1    |
|[ı, adore, coffee, #favorite]               |1.0       |1    |
|[ı, adore, rock, music, #brilliant]         |1.0       |1    |
|[ı, adore, skiing, #loveit]                 |1.0       |1    |
|[ı, adore, summer, #favorite]               |1.0       |1    |
|[ı, adore, summer, #toptastic]         